En este archivo puedes escribir lo que estimes conveniente. Te recomendamos detallar tu solución y todas las suposiciones que estás considerando. Aquí puedes ejecutar las funciones que definiste en los otros archivos de la carpeta src, medir el tiempo, memoria, etc.

In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
file_path = "farmers-protest-tweets-2021-2-4.json"

In [3]:
from q1_time import q1_time

In [5]:
q1_time(file_path)

Hola


In [6]:
import sys
!{sys.executable} -m pip install -r ../requirements.txt

In [8]:
from memory_profiler import profile